# CIFAR Experiments

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision as tv
import sys
import torch.nn as nn
import torch.nn.functional as F

sys.path.append('../../')
sys.path.append('../../src/')

import src.general as general
import src.interfaces.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune
import src.compression.quantization as quant


Files already downloaded and verified
Files already downloaded and verified


Found cached dataset imagenet-1k (/workspace/volume/cache/imagenet-1k/default-212aff79ee65f848/1.0.0/a1e9bfc56c3a7350165007d1176b15e9128fcaf9ab972147840529aed3ae52bc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset = data.supported_datasets["CIFAR-10"]

## ResNet

In [4]:
resnet = torch.load("/workspace/volume/models/resnet_cifar_finetuned.pt")

In [9]:
results = eval.get_results(resnet, dataset)

Test: 100%|██████████| 157/157 [00:18<00:00,  8.33it/s]


Test loss: 3.4313
Test score: 89.4705


In [6]:
plot.print_results(**results)

==================================== RESULTS ===================================
Loss: 0.195277
Score: 94.297373
Time per data point: 8.6058 ms
Model Size: 90.04 MB
Number of parameters: 23528522
Number of MACs: 4119896576


In [12]:
general.finetune(resnet, dataset, 99, save_path="/workspace/volume/models/resnet_cifar.pt", patience=1)

Validate: 100%|██████████| 79/79 [00:09<00:00,  8.52it/s]


Test loss: 0.1578
Test score: 95.8465


Train: 100%|██████████| 704/704 [02:44<00:00,  4.28it/s]


Train loss: 0.0331
Train score: 98.9569


Validate: 100%|██████████| 79/79 [00:09<00:00,  8.53it/s]


...

Train: 100%|██████████| 704/704 [02:43<00:00,  4.32it/s]


Train loss: 0.0253
Train score: 99.2143


Validate: 100%|██████████| 79/79 [00:09<00:00,  8.54it/s]

Test loss: 0.1225
Test score: 96.2619
Finetuning stopped due to early stopping with patience = 1
Finetuning finished after 3 iterations
Best score: 96.3608


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## VGG

In [14]:
vgg = torch.load("/workspace/volume/models/vgg16_cifar10.pt")

In [15]:
results = eval.get_results(vgg, dataset)
plot.print_results(**results)

Test: 100%|██████████| 157/157 [00:25<00:00,  6.09it/s]


Test loss: 0.2740
Test score: 91.9885
==================================== RESULTS ===================================
Loss: 0.274050
Score: 91.988455
Time per data point: 10.2705 ms
Model Size: 512.33 MB
Number of parameters: 134301514
Number of MACs: 15499459200


In [16]:
general.finetune(vgg, dataset, 99, save_path="/workspace/volume/models/cifar10/vgg16_cifar10.pt", patience=1)

Validate: 100%|██████████| 79/79 [00:12<00:00,  6.14it/s]


Test loss: 0.0848
Test score: 96.8354


Train: 100%|██████████| 704/704 [04:31<00:00,  2.59it/s]


Train loss: 0.0642
Train score: 97.7939


Validate: 100%|██████████| 79/79 [00:12<00:00,  6.15it/s]


...

Train: 100%|██████████| 704/704 [04:30<00:00,  2.61it/s]


Train loss: 0.0505
Train score: 98.2244


Validate: 100%|██████████| 79/79 [00:12<00:00,  6.16it/s]


Test loss: 0.0317
Test score: 99.0902
Finetuning stopped due to reaching the target score
Finetuning finished after 2 iterations
Best score: 99.0902


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## EfficientNet

In [4]:
efficientnet = torch.load("/workspace/volume/models/efficientnet_cifar10.pt")

In [ ]:
results = eval.get_results(efficientnet, dataset)
plot.print_results(**results)

Test: 100%|██████████| 157/157 [00:38<00:00,  4.12it/s]


Test loss: 0.1454
Test score: 95.9793
==================================== RESULTS ===================================
Loss: 0.145355
Score: 95.979299
Time per data point: 15.1636 ms
Model Size: 244.96 MB
Number of parameters: 63812570
Number of MACs: 5264522736


In [5]:
general.finetune(efficientnet, dataset, 99, save_path="/workspace/volume/models/cifar10/efficientnet_cifar10.pt", patience=1)

Validate: 100%|██████████| 79/79 [00:19<00:00,  4.09it/s]


Test loss: 0.1387
Test score: 96.2816


Train: 100%|██████████| 5625/5625 [10:56<00:00,  8.57it/s]


Train loss: 0.0396
Train score: 98.6978


Validate: 100%|██████████| 79/79 [00:18<00:00,  4.34it/s]


...

Train: 100%|██████████| 5625/5625 [13:34<00:00,  6.91it/s]


Train loss: 0.0287
Train score: 99.0689


Validate: 100%|██████████| 79/79 [00:28<00:00,  2.82it/s]

Test loss: 0.1242
Test score: 96.7366
Finetuning stopped due to early stopping with patience = 1
Finetuning finished after 2 iterations
Best score: 97.0530


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormAct